In [1]:
from pymeasure.instruments.newport import ESP300
from pymeasure.instruments import list_resources

from pymeasure.display import widgets as pw
from pymeasure.display.inputs import *

import ftd2xx  # Thorlabs MFF101
import ftd2xx.defines as constants
from ftd2xx import listDevices, getDeviceInfoDetail

import pymeasure.experiment.parameters as pars

from time import sleep
from pymeasure.experiment import Procedure
from pymeasure.experiment import IntegerParameter

In [ ]:
params = pars.FloatParameter(ps,units='mm')

In [ ]:
pars.ListParameter(ps)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
ctrl.FloatInput.set_parameter()

In [ ]:
pymeasure.display.inputs

In [ ]:
ps = x.position

In [2]:
resources = list_resources()

C:\Users\labuser\Anaconda3\envs\controller\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


0 : USB0::0x1313::0x8072::P2010852::INSTR : Thorlabs,PM100USB,P2010852,1.6.0
1 : ASRL1::INSTR : Not known
2 : ASRL3::INSTR : Not known
3 : ASRL6::INSTR :  
4 : GPIB0::3::INSTR : ESP300 Version 3.08 09/09/02


In [3]:
FTDIresources = listDevices()
FTDIresources

[b'37000805', b'18031043']

In [ ]:
getDeviceInfoDetail(1)

## Laser on/off

In [4]:
serial = FTDIresources[0]

In [5]:
def laser(switch):
    """Switch 'on' or 'off'"""
    serial = FTDIresources[0]
    # Raw byte commands for "MGMSG_MOT_MOVE_JOG".
#     on = b"\x6A\x04\x00\x01\x21\x01"  # x01 up
#     off = b"\x6A\x04\x00\x02\x21\x01"  # x02 down
    
    if switch == 'on':
        motor = ftd2xx.openEx(serial)
        print(motor.getDeviceInfo())
        motor.setBaudRate(115200)
        motor.setDataCharacteristics(constants.BITS_8, constants.STOP_BITS_1, constants.PARITY_NONE)
        sleep(.05)
        motor.purge()
        sleep(.05)
        motor.resetDevice()
        motor.setFlowControl(constants.FLOW_RTS_CTS, 0, 0)
        motor.setRts()

        # Send raw bytes to USB driver.
        motor.write(b"\x6A\x04\x00\x01\x21\x01")  # up or 
        motor.close()
    else:
        motor = ftd2xx.openEx(serial)
        print(motor.getDeviceInfo())
        motor.setBaudRate(115200)
        motor.setDataCharacteristics(constants.BITS_8, constants.STOP_BITS_1, constants.PARITY_NONE)
        sleep(.05)
        motor.purge()
        sleep(.05)
        motor.resetDevice()
        motor.setFlowControl(constants.FLOW_RTS_CTS, 0, 0)
        motor.setRts()

        # Send raw bytes to USB driver.
        motor.write(b"\x6A\x04\x00\x02\x21\x01")  # up or 
        motor.close()

In [6]:
laser('off')

{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}


## XYZ Control

In [7]:
ctrl = ESP300("GPIB0::3::INSTR")

In [8]:
ctrl.data_bits=8
ctrl.baud_rate=19200
ctrl.StopBits=1
ctrl.read_termination='\r\n'
ctrl.write_termination='\r'

In [9]:
x = ctrl.x  # Our x-axis is motor 1 or y
y = ctrl.y  # Our y-axis is motor 2 or x
z = ctrl.phi  # 3

In [25]:
pos = [x.position, y.position, z.position]
pos

[4.99999, 3.99999, 4.27362]

In [26]:
x.velocity = 0.1
laser('on')
x.wait(500)
x.position = 4.5
while not x.motion_done:
    sleep(.05)
# x.wait_for_stop()
laser('off')
x.position

{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}
{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}


4.50013

In [ ]:
ctrl.shutdown

### GUI

In [ ]:
from time import sleep
from pymeasure.experiment import Procedure
from pymeasure.experiment import IntegerParameter

In [ ]:
class SimpleProcedure(Procedure):

    # a Parameter that defines the number of loop iterations
    iterations = IntegerParameter('Loop Iterations')

    # a list defining the order and appearance of columns in our data file
    DATA_COLUMNS = ['Iteration']

    def execute(self):
        """ Loops over each iteration and emits the current iteration,
        before waiting for 0.01 sec, and then checking if the procedure
        should stop
        """
        for i in range(self.iterations):
            self.emit('results', {'Iteration': i})
            sleep(0.01)
            if self.should_stop():
                break

In [ ]:
procedure = SimpleProcedure()
procedure.iterations = 100

In [ ]:
from pymeasure.experiment import Results

data_filename = 'example.csv'
results = Results(procedure, data_filename)

In [ ]:
from pymeasure.experiment import Worker

worker = Worker(results)

In [ ]:
worker.start()

In [ ]:
worker.join(timeout=3600) # wait at most 1 hr (3600 sec)

In [ ]:
import logging
log = logging.getLogger(__name__)
log.addHandler(logging.NullHandler())

from time import sleep
from pymeasure.log import console_log
from pymeasure.experiment import Procedure, Results, Worker
from pymeasure.experiment import IntegerParameter

class SimpleProcedure(Procedure):

    iterations = IntegerParameter('Loop Iterations')

    DATA_COLUMNS = ['Iteration']

    def execute(self):
        log.info("Starting the loop of %d iterations" % self.iterations)
        for i in range(self.iterations):
            data = {'Iteration': i}
            self.emit('results', data)
            log.debug("Emitting results: %s" % data)
            sleep(0.01)
            if self.should_stop():
                log.warning("Caught the stop flag in the procedure")
                break

if __name__ == "__main__":
    console_log(log)

    log.info("Constructing a SimpleProcedure")
    procedure = SimpleProcedure()
    procedure.iterations = 100

    data_filename = 'example2.csv'
    log.info("Constructing the Results with a data file: %s" % data_filename)
    results = Results(procedure, data_filename)

    log.info("Constructing the Worker")
    worker = Worker(results)
    worker.start()
    log.info("Started the Worker")

    log.info("Joining with the worker in at most 1 hr")
    worker.join(timeout=3600) # wait at most 1 hr (3600 sec)
    log.info("Finished the measurement")

In [ ]:
import logging
log = logging.getLogger(__name__)
log.addHandler(logging.NullHandler())

from pymeasure.log import console_log
from pymeasure.instruments.newport import ESP300
from pymeasure.experiment import Procedure, Results, Worker
from pymeasure.experiment import IntegerParameter, FloatParameter
from time import sleep

from pdb import set_trace

class XYProcedure(Procedure):
    
    data_pts = IntegerParameter('Data points', default=0)
    max_pos = FloatParameter('Max Position', units='mm', default=5.0)
    min_pos = FloatParameter('Min Position', units='mm', default=4.5)

    DATA_COLUMNS = ['X (mm)']

    def startup(self):
        log.info('Connecting and configuring instruments')
        self.ctrl = ESP300("GPIB0::3::INSTR")
        
#         self.ctrl.data_bits = 8
#         self.ctrl.baud_rate = 19200
#         self.ctrl.StopBits = 1
#         self.ctrl.read_termination = '\r\n'
#         self.ctrl.write_termination = '\r'
#         self.ctrl.x = x  # Our x-axis is motor 1 or y
#         self.ctrl.y = y  # Our y-axis is motor 2 or x
#         self.ctrl.phi = z  # 3
    
    def execute(self):
        positions = np.linspace(
            self.max_pos,
            self.min_pos,
            num=self.data_pts
        )
        
        for position in positions:
            log.info("Reading the position to %g mm" % position)
            self.ctrl.x.position = position
            sleep(0.1)
            
            
            self.emit('results',{
                'Position (mm)': position
            })
            sleep(0.01)
            if self.should_stop():
                log.info("User aborted the procedure")
                break
                
    def shutdown(self):
        self.ctrl.shutdown()

if __name__ == "__main__":
    set_trace()
    console_log(log)

    log.info("Constructing the XYProcedure")
    procedure = XYProcedure()
    procedure.data_pts = 10
    procedure.max_pos = 4.5
    procedure.min_pos = 5.0

    data_filename = 'test.csv'
    log.info("Constructing the Results with a data file: %s" % data_filename)
    results = Results(procedure, data_filename)

    log.info("Constructing the Worker")
    worker = Worker(results)
    worker.start()
    log.info("Started the Worker")

    log.info("Joining with the worker in at most 1 hr")
    worker.join(timeout=3600) # wait at most 1 hr (3600 sec)
    log.info("Finished the measurement")

In [ ]:
ctrl.write('1HN2,1')  # y, x grouped for simultaneous movement
ctrl.ask('TB') 

In [ ]:
ctrl.write('2HN1,2')  # TODO: can't assign group #2, out of range ?
ctrl.ask('TB')

In [ ]:
#to create new group
ctrl.write('1HX')
ctrl.write('2HN2,3')  # y, z

In [ ]:
ctrl.write('1HX')  # delete group #1

In [ ]:
ctrl.ask('HB')

In [ ]:
ctrl.ask('1HN?')

In [ ]:
ctrl.ask('1VU?')  # ask whats max group vel for instrument

In [ ]:
ctrl.ask('1HV?')  # current group vel

In [ ]:
ctrl.write('1HV0.2')

In [ ]:
ctrl.ask('1HP?')  # current group pos

In [ ]:
ctrl.write('2HO')

TODO:  move both xy axes simultaneously; currently exceeds threshold.

In [ ]:
ctrl.write('1HL8,4')
ctrl.ask('TB') 

In [ ]:
x.enabled

In [ ]:
x.velocity  # get current scanning speed

In [ ]:
x.velocity = 0.4  # set scanning speed

In [ ]:
x.position  # get position

Problem: when xy axes assigned to group #1, single axes movement not allowed.
Solution: after XY motion done, delete group.


In [ ]:
x.position = 8 # move to position  
ctrl.ask('TB') 